In [1]:
import pandas as pd
from quant.logging_backtest import logger
from plotly import graph_objs as go, offline as py
from datetime import datetime

ImportError: No module named 'quant.logging_backtest'

# 复习Class部分知识点

In [56]:
class GrandPa:
    def __init__(self):
        self.x = 0
        print('I\'m GrandPa')


class Father(GrandPa):
    def __init__(self):
        print('I\'m Father!')

class Son(Father):
    """A simple example class"""
    i = 12345
    def __init__(self):
        print('这是构造函数,son')
        print (self.i)
    def sayHello(self):
        print ('hello world')

if __name__ == '__main__':
    son = Son()
    # 类型帮助信息 
    print('类型帮助信息: ',Son.__doc__)
    #类型名称
    print('类型名称:',Son.__name__)
    #类型所继承的基类
    print('类型所继承的基类:',Son.__bases__)
    #类型字典
    print('类型字典:',Son.__dict__)
    #类型所在模块
    print('类型所在模块:',Son.__module__)
    #实例类型
    print('实例类型:',Son().__class__)

SyntaxError: invalid syntax (<ipython-input-56-96a95890e385>, line 18)

In [57]:
son = Son()

这是构造函数,son
12345


In [63]:
son.sayHello()

hello world


In [64]:
grandpa = GrandPa()

I'm GrandPa


In [29]:
grandpa.x

0

In [31]:
Son.sayHello

<function __main__.Son.sayHello>

In [10]:
grandpa.__name__

AttributeError: 'GrandPa' object has no attribute '__name__'

In [12]:
class BaseClassA():
    pass
class BaseClassB():
    pass

In [18]:
class MyClass(BaseClassA, BaseClassB): 
    def __init__(self, x=1, y=0): 
        self.x = x 
        self.y = y 

In [19]:
myClass = MyClass()

In [20]:
print (myClass.x, myClass.y)

1 0


In [21]:
myClass.z = 10

In [22]:
myClass.z

10

In [23]:
print (myClass.z)
del myClass.x

10


In [24]:
myClass.x

AttributeError: 'MyClass' object has no attribute 'x'

# Wrap up in a summary 以便接手时更方便

## 所需的一些functions

In [ ]:
# 第一步：定义所有所需函数：

# 直接从SQL数据库生成资产价格dataframe
def two_assts_from_sql(asset1, asset2, sql_limit_num):
    conn = psycopg2.connect(database="bitmexdata", user="postgres", password="tongKen123", host="128.199.97.202", port="5432")
    asset1 = """ SELECT time, open, high, low, close, volume from {} order by id desc limit {}""".format(asset1, sql_limit_num)
    asset2 = """ SELECT time, open, high, low, close, volume from {} order by id desc limit {}""".format(asset2, sql_limit_num)
    df1 = pd.read_sql(asset1, con=conn)
    df2 = pd.read_sql(asset2, con=conn)
    conn.close()
    return df1, df2

# 转变数据周期：T/5T/15T/30T/H/2H/4H/D/W/M
def resample(df, period):
    convrted_df = df.resample(period).last()
    convrted_df['open'] = df['open'].resample(period).first()
    convrted_df['high'] = df['high'].resample(period).max()
    convrted_df['low'] = df['low'].resample(period).min()
    convrted_df['close'] = df['close'].resample(period).last()
    convrted_df['volume'] = df['volume'].resample(period).sum()
    # Keep rows with at least 5 non-NaN values
    convrted_df.dropna(thresh=5, inplace=True)
    convrted_df.index = convrted_df['time']
    convrted_df['time'] = pd.DatetimeIndex(convrted_df['time'])
    return convrted_df

# 为能够plot蜡烛图做timestamps处理的准备：
def cnvrt_date(convrted_df):
    cnvrted_date_df = convrted_df.copy()
    cnvrted_date_df['date'] = mpd.date2num(cnvrted_date_df['time'].dt.to_pydatetime())
    return cnvrted_date_df

# 计算两个资产的相对差累计值：
def two_assets_tmsum(cnvrted_date_df1, cnvrted_date_df2, N4):
    cnvrted_date_df1['close_shifted'] = cnvrted_date_df1['close'].shift(1)
    cnvrted_date_df2['close_shifted'] = cnvrted_date_df2['close'].shift(1)
    T1 = cnvrted_date_df1['close'].diff()/cnvrted_date_df1['close_shifted']
    T2 = cnvrted_date_df2['close'].diff()/cnvrted_date_df2['close_shifted']
    TM = T1*(1-N4/100) - T2*(N4/100)
    tmsum_sr = TM.cumsum()
    return tmsum_sr
    
# 计算相对差的移动平均快慢线：
def MAs_of_tmsum(tmsum_sr, N1, N2):
    MA1 = tmsum_sr.ewm(span= N1).mean() #快线
    MA2 = tmsum_sr.ewm(span= N2).mean() #慢线   
    return MA1, MA2
    
# Plot TMSUM图线：
# def plot_tmsum(tmsum_sr):
#     py_offline.init_notebook_mode()
#     tmsum_df = pd.DataFrame(tmsum_sr, columns=['tmsum'])
#     tmsum_df = go.Scatter(x = tmsum_df.index,
#                           y = tmsum_df['tmsum'])
#     data = [tmsum_df]
#     return py_offline.iplot(data, filename='TMSUM')

def plot_tmsum(tmsum_sr):
#     py_offline.init_notebook_mode()
    tmsum_df = pd.DataFrame(tmsum_sr, columns=['tmsum'])
    tmsum_df = go.Scatter(x = tmsum_df.index,
                          y = tmsum_df['tmsum'])
    data = [tmsum_df]
    fig = go.Figure(data=data)
    return py.iplot(fig, filename='TMSUM')
    
    
def plot_tmsum_MAs(MA1, MA2):
    # Here I didn't use offline's version of plotly, going forward  
    # will need to be consistent when moving to pycharm for plotting
    ma1_df = pd.DataFrame(MA1, columns=['MA1'], index = MA1.index)
    ma2_df = pd.DataFrame(MA2, columns=['MA2'], index = MA2.index)
    trace1 = go.Scatter(x = ma1_df.index, 
                        y = ma1_df['MA1'])
    trace2 = go.Scatter(x = ma2_df.index, 
                        y = ma2_df['MA2'])
    data = [trace1, trace2]
    fig = go.Figure(data=data)
    return py.iplot(fig, filename='plot_tmsum_MAs')
    
    
# Plot 资产实时图

def plot_candlestick(cnvrted_date_df):
    candle_df = go.Candlestick(x = cnvrted_date_df.index,
                               open = cnvrted_date_df['open'],
                               high = cnvrted_date_df['high'],
                               low  = cnvrted_date_df['low'],
                               close = cnvrted_date_df['close'])
    data = [candle_df]
    fig =go.Figure(data=data)
    return py.iplot(fig, filename='CandleStick')

# def plot_candlestick(cnvrted_date_df):
#     py_offline.init_notebook_mode()
#     candle_df = go.Candlestick(x = cnvrted_date_df.index,
#                                open = cnvrted_date_df['open'],
#                                high = cnvrted_date_df['high'],
#                                low  = cnvrted_date_df['low'],
#                                close = cnvrted_date_df['close'])
#     data = [candle_df]
#     return py_offline.iplot(data, filename='Candle Stick', image_width=400, image_height=200)



# Build a function that plots charts of two moving averages with their crossover trading signals 
# Here we want to make sure that both MA1_sr and MA2_sr are: series with timestanmps as their indexes

def MA_crossover_plot_signals(MA1_sr, MA2_sr):
    # 构建一个由 MA1和 MA2构成的dataframe：
    ma_signal_df = pd.DataFrame(MA1_sr, columns=['MA1'], index= MA1_sr.index)
    ma_signal_df['MA2'] = MA2_sr
    # 用两者的差表示 MA1在位置上高于还是低于MA2，负值说明低于，正值说明高于：
    ma_signal_df['MA1_mns_MA2'] = ma_signal_df['MA1'] - ma_signal_df['MA2']
    # 将正负值同义转换为二元的 1或者 -1便于观察和处理
    ma_signal_df['signs'] = ma_signal_df['MA1_mns_MA2']*abs(1/(ma_signal_df['MA1_mns_MA2']))
    # 用当前值和前一个周期的值决定当前状态是金叉信号还是死叉信号
    ma_signal_df['pre_signs'] = ma_signal_df['signs'].shift(1)
    ma_signal_df['signals'] = ma_signal_df['signs'] - ma_signal_df['pre_signs']
    ma_signal_df['signals_alert'] = ma_signal_df['signals'].apply(lambda x: "金叉" if x==2 else "死叉" if x==-2 else "无信号")
    # 将所有出现信号的rows挑出来建立一个字典：
    ma_df_with_signals = ma_signal_df[ma_signal_df['signals_alert'] != "无信号"]['signals_alert']
    signal_dict = dict(ma_df_with_signals)
    # Add each and all signal information(dictionary format) to annotations(list format) 
    # so that it can be put into the go.Layout() function.
    annotations = []
    each_dict = {}
    for i, k in enumerate(signal_dict):
        each_dict['x'] = k
        each_dict['y'] = ma_signal_df['MA1'].loc[k]
        each_dict['text']=ma_signal_df['signals_alert'].loc[k]
        each_dict['showarrow']=True
        each_dict['arrowhead']=7
        each_dict['ax']=0
        each_dict['ay']=-40

        annotations.append(each_dict.copy())
    # 用plotly进行绘图，包括了之前处理好的annotations，作为显示信号的功能
    ma1_df = pd.DataFrame(tmsum_ma1, columns=['MA1'], index = tmsum_ma1.index)
    ma2_df = pd.DataFrame(tmsum_ma2, columns=['MA2'], index = tmsum_ma2.index)
    trace1 = go.Scatter(x = ma1_df.index, 
              y = ma1_df['MA1'])
    trace2 = go.Scatter(x = ma2_df.index, 
              y = ma2_df['MA2'])
    layout = go.Layout(
        showlegend=False,
        annotations=annotations
    )
    data = [trace1, trace2]
    fig = go.Figure(data=data, 
                    layout=layout)
    return py.iplot(fig, filename='plot_MAs_with_signals')
#     return plotly.offline.plot(fig, filename = 'test.html') # 如果要用html那么则会以网页的形式输出图像

## 从SQL数据库里调用数据，做一个资产的完整dataframe出来，为plotting做准备

In [65]:
def asst_from_sql(asset, sql_limit_num):
    conn = psycopg2.connect(database="bitmexdata", user="postgres", password="tongKen123", host="128.199.97.202", port="5432")
    asset = """ SELECT time, open, high, low, close, volume from {} order by id desc limit {}""".format(asset, sql_limit_num)
    df = pd.read_sql(asset, con=conn)
    conn.close()
    return df

In [ ]:
eth_sql = asst_from_sql('ethz18_1m')

In [ ]:
class Plotter(PlotBase):
    
    def __init__(self, instrument, bar, fill, context):
        super().__init__()

        self.bar = bar.total_dict
        self.equity_df = fill.equity.df
        self.cash_df = fill.cash.df
        self.position_df = fill.position.df
        self.realized_G_L_df = fill.realized_gain_and_loss.df
        self.unrealized_g_l_df = fill.unrealized_gain_and_loss.df
        self.commission_df = fill.commission.df
        df = context.trade_log
        cash = context.initial_cash
        self.income_df = (df[df['position'] == 0]['equity'] - cash).to_frame()  # 损益
        self.data = []
        self.update_menus = []
        self.data_dict = {
            '权益': self.equity_df,
            '现金': self.cash_df,
            '平仓盈亏': self.realized_G_L_df,
            '浮动盈亏': self.unrealized_g_l_df,
            '损益': self.income_df,

    # Brad新加的部分从这里开始：
    def resample(df, period):
        convrted_df = df.resample(period).last()
        convrted_df['open'] = df['open'].resample(period).first()
        convrted_df['high'] = df['high'].resample(period).max()
        convrted_df['low'] = df['low'].resample(period).min()
        convrted_df['close'] = df['close'].resample(period).last()
        convrted_df['volume'] = df['volume'].resample(period).sum()
        # Keep rows with at least 5 non-NaN values
        convrted_df.dropna(thresh=5, inplace=True)
        convrted_df.index = convrted_df['time']
        convrted_df['time'] = pd.DatetimeIndex(convrted_df['time'])
        return convrted_df       

    # 为能够plot蜡烛图做timestamps处理的准备：
    def cnvrt_date(convrted_df):
        cnvrted_date_df = convrted_df.copy()
        cnvrted_date_df['date'] = mpd.date2num(cnvrted_date_df['time'].dt.to_pydatetime())
        return cnvrted_date_df
            
    def plot_candlestick(asset1, asset2, sql_limit_num, period):
        # 直接从SQL数据库生成资产价格dataframe
        conn = psycopg2.connect(database="bitmexdata", user="postgres", password="tongKen123", host="128.199.97.202", port="5432")
        asset1 = """ SELECT time, open, high, low, close, volume from {} order by id desc limit {}""".format(asset1, sql_limit_num)
        asset2 = """ SELECT time, open, high, low, close, volume from {} order by id desc limit {}""".format(asset2, sql_limit_num)
        df1 = pd.read_sql(asset1, con=conn)
        df2 = pd.read_sql(asset2, con=conn)
        conn.close()            
            
        res_asst1 = resample(df1, period)
        res_asst2 = resample(df2, period)
            
            
            
    # 直接从SQL数据库生成资产价格dataframe
    def two_assts_from_sql(asset1, asset2, sql_limit_num):
        conn = psycopg2.connect(database="bitmexdata", user="postgres", password="tongKen123", host="128.199.97.202", port="5432")
        asset1 = """ SELECT time, open, high, low, close, volume from {} order by id desc limit {}""".format(asset1, sql_limit_num)
        asset2 = """ SELECT time, open, high, low, close, volume from {} order by id desc limit {}""".format(asset2, sql_limit_num)
        df1 = pd.read_sql(asset1, con=conn)
        df2 = pd.read_sql(asset2, con=conn)
        conn.close()
        return df1, df2
    
    # 转变数据周期：T/5T/15T/30T/H/2H/4H/D/W/M


# 第一部分：Candlestick

In [ ]:
class Plot_CandleStick_And_Signals():
    
    # initialization的部分：
    def 
        conn = psycopg2.connect(database="bitmexdata", user="postgres", password="tongKen123", host="128.199.97.202", port="5432")
        asset1 = """ SELECT time, open, high, low, close, volume from {} order by id desc limit {}""".format(asset1, sql_limit_num)
        asset2 = """ SELECT time, open, high, low, close, volume from {} order by id desc limit {}""".format(asset2, sql_limit_num)
        df1 = pd.read_sql(asset1, con=conn)
        df2 = pd.read_sql(asset2, con=conn)
        conn.close()    

    def resample(df, period):
        convrted_df = df.resample(period).last()
        convrted_df['open'] = df['open'].resample(period).first()
        convrted_df['high'] = df['high'].resample(period).max()
        convrted_df['low'] = df['low'].resample(period).min()
        convrted_df['close'] = df['close'].resample(period).last()
        convrted_df['volume'] = df['volume'].resample(period).sum()
        # Keep rows with at least 5 non-NaN values
        convrted_df.dropna(thresh=5, inplace=True)
        convrted_df.index = convrted_df['time']
        convrted_df['time'] = pd.DatetimeIndex(convrted_df['time'])
        return convrted_df  
    
    # 直接从SQL数据库生成资产价格dataframe
    def two_assts_from_sql(asset1, asset2, sql_limit_num):

        return df1, df2
    
    
    
    
class Plot_Signals():
    def 